In [ ]:
# Import standard and third-party libraries used throughout this notebook
# - load_dotenv: loads environment variables from a .env file into os.environ
# - requests: used to make HTTP POST requests to the Pushover API
# - os: used to read environment variables once loaded
# - json: used when serializing/deserializing arguments for tool calls
from dotenv import load_dotenv  # load environment variables from a .env file
import requests                # HTTP client for sending requests to Pushover
import os                      # access environment variables
import json                    # parse/produce JSON for tool argument handling


In [ ]:
# Load environment variables from a .env file located next to this notebook
# Make sure to run this cell before using environment-dependent variables
load_dotenv(override=True)

# Retrieve Pushover credentials from the environment
pushover_user = os.getenv("PUSHOVER_USER")   # Pushover user key (recipient)
pushover_token = os.getenv("PUSHOVER_TOKEN") # Pushover application token (sender)

# Pushover API endpoint for sending messages
pushover_url = "https://api.pushover.net/1/messages.json"

# Debug prints to verify values during development (remove in production)
print("pushover_user:", bool(pushover_user), "pushover_token:", bool(pushover_token))


In [ ]:
# Define a helper function to send a message via Pushover
# - Accepts a text message and posts it to the Pushover API using stored credentials
# - Prints the message for local debugging and returns the response object for inspection

def push(message: str):
    # Show the message locally so you can confirm in notebook output
    print(f"Push: {message}")

    # Prepare the form-encoded payload the Pushover API expects
    # Required keys: token (app token), user (recipient key), message (text)
    payload = {
        "token": pushover_token,
        "user": pushover_user,
        "message": message
    }

    # Send the POST request and return the Response for debugging
    response = requests.post(pushover_url, data=payload)
    print("Pushover response status:", response.status_code)
    try:
        print("Pushover response body:", response.json())
    except Exception:
        print("Pushover response text:", response.text)
    return response


In [ ]:
# Test sending a simple message with a timestamp to verify notifications work
from datetime import datetime

# Compose a human-readable message including the current timestamp
message = f"Hello android from arch - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Call the push helper to send the test notification
push(message)


Push: Hello android from arch - 2025-09-29 18:05:12


In [ ]:
# Helper function to record user interest details and notify via Pushover
# - email: required email address string
# - name: optional display name
# - notes: optional additional notes
# Returns a simple dict acknowledging the recording

def record_user_details(email: str, name: str = "Name not provided", notes: str = "not provided"):
    # Send a push notification summarizing the recorded data
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}


# Helper function to record unknown questions so they can be reviewed later
# - question: the text of the user question that wasn't answered
# Returns a simple dict acknowledging the recording

def record_unknown_question(question: str):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}


In [ ]:
# JSON Schema describing the 'record_user_details' function for external tooling
# This can be consumed by automation frameworks or agent components to know
# what parameters are required and their types.
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}


# JSON Schema describing the 'record_unknown_question' function for external tooling
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}


In [ ]:
# A list of tool descriptors that can be provided to an agent or automation system.
# Each entry describes a function-type tool and includes the JSON schema defined above.
tools = [
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_question_json}
]

# This 'tools' list is often passed into an agent constructor so it knows what
# tools are available and how to call them programmatically.


In [ ]:
# Function to handle incoming tool call objects and dispatch to the matching helper
# This version shows an explicit if/elif dispatch for clarity and safety.
# tool_calls is expected to be an iterable of objects with attributes:
# - function.name: the name of the tool function to call
# - function.arguments: a JSON string of the arguments to pass
# - id: an identifier for the tool call

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        # Extract the intended tool name and arguments from the tool_call object
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # Dispatch to the appropriate function using explicit checks
        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
        else:
            result = {"error": f"unknown tool {tool_name}"}

        # Append the tool result in the expected format
        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results


# Example demonstrating direct invocation from the global namespace
# Useful for quick tests: calls record_unknown_question with a sample question
globals()["record_unknown_question"]("KeyError?")


# Alternative dynamic dispatch implementation (commented-out)
# This approach uses globals().get(tool_name) to find and call the function
# It is more dynamic but can be less explicit for code readers and static analyzers.
#
# def handle_tool_calls_dynamic(tool_calls):
#     results = []
#     for tool_call in tool_calls:
#         tool_name = tool_call.function.name
#         arguments = json.loads(tool_call.function.arguments)
#         print(f"Tool called: {tool_name}", flush=True)
#         tool = globals().get(tool_name)
#         if tool:
#             result = tool(**arguments)
#         else:
#             result = {"error": "unknown tool"}
#         results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
#     return results


In [12]:
# This calls the function 'record_unknown_question' from the global scope with the argument "this is a really hard question"
globals()["record_unknown_question"]("KeyError?")

Push: Recording KeyError? asked that I couldn't answer


{'recorded': 'ok'}

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return result